In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
half = "2H"
max_min = 30
min_min = max_min - 15
max_sec = max_min*60
min_sec = min_min*60
table_name = "final_in_poss_df"

In [3]:
print(f'{table_name}_{half}_{min_min}_{max_min}.csv')

final_in_poss_df_2H_15_30.csv


In [4]:
with open("../../events/events_England.json", "r") as file:
    event_data_2 = json.load(file)

df2 = pd.DataFrame(event_data_2)
df2 = df2[df2.matchPeriod == half]
df2 = df2[df2.eventSec >= min_sec]
if max_min < 45:
    df2 = df2[df2.eventSec < max_sec]

with open("../../events/events_France.json", "r") as file:
    event_data_3 = json.load(file)

df3 = pd.DataFrame(event_data_3)
df3 = df3[df3.matchPeriod == half]
df3 = df3[df3.eventSec >= min_sec]
if max_min < 45:
    df3 = df3[df3.eventSec < max_sec]


with open("../../events/events_Germany.json", "r") as file:
    event_data_4 = json.load(file)

df4 = pd.DataFrame(event_data_4)
df4 = df4[df4.matchPeriod == half]
df4 = df4[df4.eventSec >= min_sec]
if max_min < 45:
    df4 = df4[df4.eventSec < max_sec]


with open("../../events/events_Italy.json", "r") as file:
    event_data_5 = json.load(file)

df5 = pd.DataFrame(event_data_5)
df5 = df5[df5.matchPeriod == half]
df5 = df5[df5.eventSec >= min_sec]
if max_min < 45:
    df5 = df5[df5.eventSec < max_sec]


with open("../../events/events_Spain.json", "r") as file:
    event_data_6 = json.load(file)

df6 = pd.DataFrame(event_data_6)
df6 = df6[df6.matchPeriod == half]
df6 = df6[df6.eventSec >= min_sec]
if max_min < 45:
    df6 = df6[df6.eventSec < max_sec]


with open("../../events/teams.json", "r") as file:
    teams_data = json.load(file)

teams_df = pd.DataFrame(teams_data)

In [5]:
all_events_df = pd.concat([df2, df3, df4, df5, df6], ignore_index=True)

# Feature engineering - Pass DF - Pass Motifs

In [6]:
passes_df = all_events_df[all_events_df['eventName'] == 'Pass']

# Concatenating teamId and matchId to create a unique identifier for each team in each match
passes_df['uniqueTeamId'] = passes_df['teamId'].astype(str) + "---" + passes_df['matchId'].astype(str)

<ipython-input-6-846069bd07cf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['uniqueTeamId'] = passes_df['teamId'].astype(str) + "---" + passes_df['matchId'].astype(str)


In [7]:
# Function to classify pass motifs
def classify_pass_motif(passes):
    players = passes['playerId'].values
    if len(set(players)) == 2:  # ABAB
        return 'ABAB'
    elif len(set(players)) == 3:
        if players[0] == players[2] or players[1] == players[3]:  # ABCA or ABCB
            return 'ABCA' if players[0] == players[2] else 'ABCB'
        else:
            return 'ABCD'  # ABCD for distinct players or other scenarios
    return 'ABCD'

# Apply the classification within a rolling window
def apply_motif_classification(df):
    # Ensure the DataFrame is sorted by event time
    df = df.sort_values(by='eventSec')
    motifs = []
    # Using a rolling window of 4 events
    for i in range(len(df) - 3):
        window = df.iloc[i:i+4]
        motif = classify_pass_motif(window)
        motifs.append((window.iloc[-1]['uniqueTeamId'], motif))
    return pd.DataFrame(motifs, columns=['uniqueTeamId', 'Motif'])

In [8]:
motif_classification_results = passes_df.groupby('uniqueTeamId').apply(apply_motif_classification)

In [9]:
motif_classification_results.reset_index(drop=True, inplace=True)

In [10]:
motif_counts = motif_classification_results.groupby(['uniqueTeamId', 'Motif']).size().unstack(fill_value=0)

In [11]:
motif_ratios = motif_counts.div(motif_counts.sum(axis=1), axis=0)
# df to be used in clustering

In [12]:
motif_ratios

Motif,ABAB,ABCA,ABCB,ABCD
uniqueTeamId,,,,
10531---2499726,0.000000,0.108108,0.108108,0.783784
10531---2499736,0.034483,0.086207,0.086207,0.793103
10531---2499741,0.030303,0.060606,0.060606,0.848485
10531---2499757,0.055556,0.088889,0.100000,0.755556
10531---2499766,0.000000,0.000000,0.000000,1.000000
...,...,...,...,...
756---2565880,0.034483,0.103448,0.103448,0.758621
756---2565892,0.015625,0.093750,0.078125,0.812500
756---2565904,0.037037,0.172840,0.172840,0.617284


In [13]:
motif_stats = motif_ratios.agg(['mean', 'std']).T

In [14]:
motif_stats

,mean,std
Motif,,
ABAB,0.035882,0.033758
ABCA,0.108919,0.040440
ABCB,0.109377,0.040897
ABCD,0.745821,0.089116


# Feature engineering - Shot DF

In [15]:
shot_df = all_events_df[all_events_df['eventName'] == 'Shot']
shot_df['uniqueTeamId'] = shot_df['teamId'].astype(str) + "---" + shot_df['matchId'].astype(str)

<ipython-input-15-396a5f5d4462>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['uniqueTeamId'] = shot_df['teamId'].astype(str) + "---" + shot_df['matchId'].astype(str)


In [16]:
import math

def calculate_shot_distance(row):
    if row['positions'] is not None and len(row['positions']) > 0:
        loc = row['positions'][0]
        # Normalize locations
        x = loc['x'] * (120 / 100.0)  # Normalizing the x coordinate
        y = loc['y'] * (80 / 100.0)   # Normalizing the y coordinate
        # Calculate Euclidean distance from (0,0)
        distance = math.sqrt((120-x)**2 + (80-y)**2)
        return distance
    return None

# Apply the function to calculate shot distance and add it to the DataFrame
shot_df['shot_distance'] = shot_df.apply(calculate_shot_distance, axis=1)

<ipython-input-16-103ed8455f10>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['shot_distance'] = shot_df.apply(calculate_shot_distance, axis=1)


In [17]:
def classify_shot_by_distance(distance):
    if distance <= 38:
        return "Near"
    elif distance <= 52:
        return "Middle Distance"
    else:
        return "Far"

# Assuming shot_df has a 'shot_distance' column calculated as before
shot_df['shot_class'] = shot_df['shot_distance'].apply(classify_shot_by_distance)

# Grouping shots by their class and counting
shot_class_counts = shot_df.groupby('uniqueTeamId')['shot_class'].value_counts().unstack(fill_value=0)
shot_counts = shot_df.groupby('uniqueTeamId')['subEventName'].value_counts().unstack(fill_value=0)

<ipython-input-17-0451b5045683>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['shot_class'] = shot_df['shot_distance'].apply(classify_shot_by_distance)


In [18]:
shot_df_final = pd.concat([shot_counts, shot_class_counts], axis=1)

In [19]:
shot_df_final

,Shot,Far,Middle Distance,Near
uniqueTeamId,,,,
10531---2499736,1,0,1,0
10531---2499741,1,1,0,0
10531---2499788,1,1,0,0
10531---2499806,2,0,1,1
10531---2499855,3,1,1,1
...,...,...,...,...
756---2565880,3,0,3,0
756---2565892,3,1,1,1
756---2565904,3,0,3,0


In [20]:
shot_df_final.rename(columns = {'Shot':'Total Shots'}, inplace = True) 

shot_df_final["Far"] = shot_df_final["Far"]/shot_df_final["Total Shots"]
shot_df_final["Middle Distance"] = shot_df_final["Middle Distance"]/shot_df_final["Total Shots"]
shot_df_final["Near"] = shot_df_final["Near"]/shot_df_final["Total Shots"]


shot_df_final.rename(columns = {'Near':'Near Shots Ratio', 'Middle Distance':'Middle Distance Shots Ratio', 'Far':'Far Shots Ratio',}, inplace = True) 

In [21]:
shot_df_final

,Total Shots,Far Shots Ratio,Middle Distance Shots Ratio,Near Shots Ratio
uniqueTeamId,,,,
10531---2499736,1,0.000000,1.000000,0.000000
10531---2499741,1,1.000000,0.000000,0.000000
10531---2499788,1,1.000000,0.000000,0.000000
10531---2499806,2,0.000000,0.500000,0.500000
10531---2499855,3,0.333333,0.333333,0.333333
...,...,...,...,...
756---2565880,3,0.000000,1.000000,0.000000
756---2565892,3,0.333333,0.333333,0.333333
756---2565904,3,0.000000,1.000000,0.000000


# Feature engineering - Pass DF - Pass Types

In [22]:
def determine_zone(position):
    if 'x' in position and 'y' in position:
        x, y = position['x'], position['y']
        zone = int(x // 33.333) + 1
        return zone
    else:
        return None
    
# Apply the function to determine zones for each event
passes_df['Zone'] = passes_df['positions'].apply(lambda pos: determine_zone(pos[0]) if len(pos) > 0 else None)

<ipython-input-22-1dcee42e8a09>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['Zone'] = passes_df['positions'].apply(lambda pos: determine_zone(pos[0]) if len(pos) > 0 else None)


In [23]:
passes_df.loc[passes_df['Zone'] == 4, ['Zone']] = 3

In [24]:
passes_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,uniqueTeamId,Zone
5,8,Simple pass,[{'id': 1801}],7945,"[{'y': 18, 'x': 61}, {'y': 44, 'x': 57}]",2499719,Pass,1609,2H,907.501291,85,177960401,1609---2499719,2
6,8,Simple pass,[{'id': 1801}],120339,"[{'y': 44, 'x': 57}, {'y': 56, 'x': 59}]",2499719,Pass,1609,2H,910.862520,85,177960403,1609---2499719,2
9,8,Simple pass,[{'id': 1801}],3319,"[{'y': 40, 'x': 59}, {'y': 41, 'x': 70}]",2499719,Pass,1609,2H,915.969053,85,177960406,1609---2499719,2
15,8,High pass,"[{'id': 1901}, {'id': 901}, {'id': 1802}]",14763,"[{'y': 60, 'x': 41}, {'y': 41, 'x': 71}]",2499719,Pass,1631,2H,920.657931,83,177960453,1631---2499719,2
17,8,Simple pass,[{'id': 1801}],370224,"[{'y': 70, 'x': 36}, {'y': 91, 'x': 62}]",2499719,Pass,1609,2H,925.342173,85,177960413,1609---2499719,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461153,8,Simple pass,"[{'id': 1901}, {'id': 1801}]",22578,"[{'y': 8, 'x': 32}, {'y': 28, 'x': 21}]",2565927,Pass,682,2H,1761.665240,85,253302258,682---2565927,1
461154,8,Simple pass,"[{'id': 1901}, {'id': 1801}]",364860,"[{'y': 28, 'x': 21}, {'y': 59, 'x': 32}]",2565927,Pass,682,2H,1764.799344,85,253302259,682---2565927,1
461155,8,Simple pass,"[{'id': 1901}, {'id': 1801}]",399676,"[{'y': 59, 'x': 32}, {'y': 42, 'x': 39}]",2565927,Pass,682,2H,1766.290679,85,253302260,682---2565927,1
461156,8,Simple pass,"[{'id': 1901}, {'id': 1801}]",267134,"[{'y': 42, 'x': 39}, {'y': 86, 'x': 57}]",2565927,Pass,682,2H,1767.186478,85,253302262,682---2565927,2


In [25]:
import math

def calculate_pass_details_from_single_column(row):
    if row['positions'] is not None and len(row['positions']) == 2:
        # Extract and normalize start and end locations
        start_loc = row['positions'][0]
        end_loc = row['positions'][1]
        
        start_x = start_loc['x'] * (120 / 100.0)
        start_y = start_loc['y'] * (80 / 100.0)
        end_x = end_loc['x'] * (120 / 100.0)
        end_y = end_loc['y'] * (80 / 100.0)
        
        # Calculate pass length and angle
        delta_x = end_x - start_x
        delta_y = end_y - start_y
        pass_length = math.sqrt(delta_x**2 + delta_y**2)
        pass_angle = math.atan2(delta_y, delta_x)
        
        return pd.Series([pass_length, pass_angle])
    else:
        return pd.Series([None, None])

In [26]:
passes_df[['calculated_pass_length', 'calculated_pass_angle']] = passes_df.apply(calculate_pass_details_from_single_column, axis=1)


<ipython-input-26-53702553e163>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df[['calculated_pass_length', 'calculated_pass_angle']] = passes_df.apply(calculate_pass_details_from_single_column, axis=1)
<ipython-input-26-53702553e163>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df[['calculated_pass_length', 'calculated_pass_angle']] = passes_df.apply(calculate_pass_details_from_single_column, axis=1)


In [27]:
def determine_pass_direction(angle):
    # Forward Pass: -π/4 < angle < π/4
    if -np.pi/4 < angle < np.pi/4:
        return "Forward Pass"
    # Backward Pass: 3π/4 < angle <= π or -π <= angle < -3π/4
    elif 3*np.pi/4 < angle <= np.pi or -np.pi <= angle < -3*np.pi/4:
        return "Backward Pass"
    # Side Pass: Otherwise
    else:
        return "Side Pass"

In [28]:
passes_df['pass_direction'] = passes_df['calculated_pass_angle'].apply(determine_pass_direction)

<ipython-input-28-a0b0c4f17a64>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['pass_direction'] = passes_df['calculated_pass_angle'].apply(determine_pass_direction)


In [29]:
passes_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,uniqueTeamId,Zone,calculated_pass_length,calculated_pass_angle,pass_direction
5,8,Simple pass,[{'id': 1801}],7945,"[{'y': 18, 'x': 61}, {'y': 44, 'x': 57}]",2499719,Pass,1609,2H,907.501291,85,177960401,1609---2499719,2,21.346663,1.797595,Side Pass
6,8,Simple pass,[{'id': 1801}],120339,"[{'y': 44, 'x': 57}, {'y': 56, 'x': 59}]",2499719,Pass,1609,2H,910.862520,85,177960403,1609---2499719,2,9.895454,1.325818,Side Pass
9,8,Simple pass,[{'id': 1801}],3319,"[{'y': 40, 'x': 59}, {'y': 41, 'x': 70}]",2499719,Pass,1609,2H,915.969053,85,177960406,1609---2499719,2,13.224220,0.060532,Forward Pass
15,8,High pass,"[{'id': 1901}, {'id': 901}, {'id': 1802}]",14763,"[{'y': 60, 'x': 41}, {'y': 41, 'x': 71}]",2499719,Pass,1631,2H,920.657931,83,177960453,1631---2499719,2,39.077359,-0.399515,Forward Pass
17,8,Simple pass,[{'id': 1801}],370224,"[{'y': 70, 'x': 36}, {'y': 91, 'x': 62}]",2499719,Pass,1609,2H,925.342173,85,177960413,1609---2499719,2,35.435575,0.493941,Forward Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461153,8,Simple pass,"[{'id': 1901}, {'id': 1801}]",22578,"[{'y': 8, 'x': 32}, {'y': 28, 'x': 21}]",2565927,Pass,682,2H,1761.665240,85,253302258,682---2565927,1,20.742227,2.260596,Side Pass
461154,8,Simple pass,"[{'id': 1901}, {'id': 1801}]",364860,"[{'y': 28, 'x': 21}, {'y': 59, 'x': 32}]",2565927,Pass,682,2H,1764.799344,85,253302259,682---2565927,1,28.094128,1.081677,Side Pass
461155,8,Simple pass,"[{'id': 1901}, {'id': 1801}]",399676,"[{'y': 59, 'x': 32}, {'y': 42, 'x': 39}]",2565927,Pass,682,2H,1766.290679,85,253302260,682---2565927,1,15.984993,-1.017502,Side Pass
461156,8,Simple pass,"[{'id': 1901}, {'id': 1801}]",267134,"[{'y': 42, 'x': 39}, {'y': 86, 'x': 57}]",2565927,Pass,682,2H,1767.186478,85,253302262,682---2565927,2,41.298910,1.020410,Side Pass


In [30]:
np.unique(passes_df.subEventName)

array(['Cross', 'Hand pass', 'Head pass', 'High pass', 'Launch',
       'Simple pass', 'Smart pass'], dtype=object)

In [31]:
passes_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,uniqueTeamId,Zone,calculated_pass_length,calculated_pass_angle,pass_direction
5,8,Simple pass,[{'id': 1801}],7945,"[{'y': 18, 'x': 61}, {'y': 44, 'x': 57}]",2499719,Pass,1609,2H,907.501291,85,177960401,1609---2499719,2,21.346663,1.797595,Side Pass
6,8,Simple pass,[{'id': 1801}],120339,"[{'y': 44, 'x': 57}, {'y': 56, 'x': 59}]",2499719,Pass,1609,2H,910.862520,85,177960403,1609---2499719,2,9.895454,1.325818,Side Pass
9,8,Simple pass,[{'id': 1801}],3319,"[{'y': 40, 'x': 59}, {'y': 41, 'x': 70}]",2499719,Pass,1609,2H,915.969053,85,177960406,1609---2499719,2,13.224220,0.060532,Forward Pass
15,8,High pass,"[{'id': 1901}, {'id': 901}, {'id': 1802}]",14763,"[{'y': 60, 'x': 41}, {'y': 41, 'x': 71}]",2499719,Pass,1631,2H,920.657931,83,177960453,1631---2499719,2,39.077359,-0.399515,Forward Pass
17,8,Simple pass,[{'id': 1801}],370224,"[{'y': 70, 'x': 36}, {'y': 91, 'x': 62}]",2499719,Pass,1609,2H,925.342173,85,177960413,1609---2499719,2,35.435575,0.493941,Forward Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461153,8,Simple pass,"[{'id': 1901}, {'id': 1801}]",22578,"[{'y': 8, 'x': 32}, {'y': 28, 'x': 21}]",2565927,Pass,682,2H,1761.665240,85,253302258,682---2565927,1,20.742227,2.260596,Side Pass
461154,8,Simple pass,"[{'id': 1901}, {'id': 1801}]",364860,"[{'y': 28, 'x': 21}, {'y': 59, 'x': 32}]",2565927,Pass,682,2H,1764.799344,85,253302259,682---2565927,1,28.094128,1.081677,Side Pass
461155,8,Simple pass,"[{'id': 1901}, {'id': 1801}]",399676,"[{'y': 59, 'x': 32}, {'y': 42, 'x': 39}]",2565927,Pass,682,2H,1766.290679,85,253302260,682---2565927,1,15.984993,-1.017502,Side Pass
461156,8,Simple pass,"[{'id': 1901}, {'id': 1801}]",267134,"[{'y': 42, 'x': 39}, {'y': 86, 'x': 57}]",2565927,Pass,682,2H,1767.186478,85,253302262,682---2565927,2,41.298910,1.020410,Side Pass


In [32]:
#passes_count = passes_df.groupby('teamId')['subEventName'].value_counts().unstack(fill_value=0)

In [33]:
# Step 1: Combine 'pass_direction' and 'Zone' into a single column
passes_df['dir_zone'] = passes_df['pass_direction'].astype(str) + '_Zone_' + passes_df['Zone'].astype(str)

# Step 2: Group by 'uniqueTeamId' and the new 'dir_zone' column, then count
direction_zone_count = passes_df.groupby(['uniqueTeamId', 'dir_zone']).size()

# Step 3: Pivot the results to have 'uniqueTeamId' as rows and each unique 'dir_zone' combination as columns
direction_zone_count_pivot = direction_zone_count.unstack(fill_value=0)

direction_count = passes_df.groupby('uniqueTeamId')['pass_direction'].value_counts().unstack(fill_value=0)

pass_direction_df = pd.concat([direction_zone_count_pivot, direction_count], axis=1)

<ipython-input-33-54e3dcb6a0f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['dir_zone'] = passes_df['pass_direction'].astype(str) + '_Zone_' + passes_df['Zone'].astype(str)


In [34]:
pass_direction_df

,Backward Pass_Zone_1,Backward Pass_Zone_2,Backward Pass_Zone_3,Forward Pass_Zone_1,Forward Pass_Zone_2,Forward Pass_Zone_3,Side Pass_Zone_1,Side Pass_Zone_2,Side Pass_Zone_3,Backward Pass,Forward Pass,Side Pass
uniqueTeamId,,,,,,,,,,,,
10531---2499726,0,5,0,11,4,0,4,14,2,5,15,20
10531---2499736,0,0,3,11,7,4,10,14,12,3,22,36
10531---2499741,1,4,1,10,10,1,4,4,1,6,21,9
10531---2499757,1,6,5,16,12,5,11,15,22,12,33,48
10531---2499766,1,0,0,3,2,0,2,1,2,1,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...
756---2565880,0,5,6,8,18,6,8,19,20,11,32,47
756---2565892,0,6,2,6,15,5,10,12,11,8,26,33
756---2565904,3,9,6,13,12,2,12,17,10,18,27,39


## Pass Types

In [35]:
high_pass_types = ['High pass', 'Cross', 'Launch']

all_pass_types = passes_df['subEventName'].unique()
low_pass_types = [pt for pt in all_pass_types if pt not in high_pass_types and pt != 'Smart pass']

# --- overall ---
pass_counts = passes_df.groupby(['uniqueTeamId', 'subEventName']).size().unstack(fill_value=0)
pass_counts['Total_Passes'] = pass_counts.sum(axis=1)

# reindex to avoid missing columns (robust)
pass_counts['High_pass_Total'] = pass_counts.reindex(columns=high_pass_types, fill_value=0).sum(axis=1)
pass_counts['Low_pass_Total']  = pass_counts.reindex(columns=low_pass_types,  fill_value=0).sum(axis=1)

pass_counts['High_pass_Ratio_Total'] = pass_counts['High_pass_Total'] / pass_counts['Total_Passes']
pass_counts['Low_pass_Ratio_Total']  = pass_counts['Low_pass_Total'] / pass_counts['Total_Passes']

final_pass_types_df = pass_counts[['Total_Passes', 'High_pass_Ratio_Total', 'Low_pass_Ratio_Total']].copy()

# --- zone-specific ---
zones = passes_df['Zone'].unique()
for zone in zones:
    zone_df = passes_df[passes_df['Zone'] == zone]

    zone_pass_counts = zone_df.groupby(['uniqueTeamId', 'subEventName']).size().unstack(fill_value=0)
    zone_pass_counts['Total_Passes'] = zone_pass_counts.sum(axis=1)

    zone_high_total = zone_pass_counts.reindex(columns=high_pass_types, fill_value=0).sum(axis=1)
    zone_low_total  = zone_pass_counts.reindex(columns=low_pass_types,  fill_value=0).sum(axis=1)

    # avoid divide-by-zero: teams with 0 passes in that zone -> NaN
    denom = zone_pass_counts['Total_Passes'].replace(0, pd.NA)

    final_pass_types_df[f'High_pass_Ratio_Zone{zone}'] = zone_high_total / denom
    final_pass_types_df[f'Low_pass_Ratio_Zone{zone}']  = zone_low_total / denom


In [36]:
final_pass_types_df

subEventName,Total_Passes,High_pass_Ratio_Total,Low_pass_Ratio_Total,High_pass_Ratio_Zone2,Low_pass_Ratio_Zone2,High_pass_Ratio_Zone3,Low_pass_Ratio_Zone3,High_pass_Ratio_Zone1,Low_pass_Ratio_Zone1
uniqueTeamId,,,,,,,,,
10531---2499726,40,0.075000,0.925000,0.000000,1.000000,0.000000,1.000000,0.200000,0.800000
10531---2499736,61,0.196721,0.786885,0.095238,0.904762,0.368421,0.578947,0.142857,0.857143
10531---2499741,36,0.333333,0.666667,0.222222,0.777778,0.000000,1.000000,0.533333,0.466667
10531---2499757,93,0.118280,0.870968,0.121212,0.878788,0.125000,0.843750,0.107143,0.892857
10531---2499766,11,0.272727,0.727273,0.333333,0.666667,0.500000,0.500000,0.166667,0.833333
...,...,...,...,...,...,...,...,...,...
756---2565880,90,0.211111,0.755556,0.142857,0.857143,0.343750,0.562500,0.125000,0.875000
756---2565892,67,0.208955,0.761194,0.151515,0.787879,0.388889,0.611111,0.125000,0.875000
756---2565904,84,0.190476,0.797619,0.131579,0.842105,0.444444,0.555556,0.107143,0.892857


# Pass_Connectivity

In [37]:
from collections import defaultdict
import networkx as nx

In [38]:
def calculate_pass_connectivity(group):
    # Initialize two empty graphs for the two teams in the match
    team_graphs = {team_id: nx.Graph() for team_id in group['uniqueTeamId'].unique()}
    
    # Track the previous player and team to identify consecutive passes
    prev_player = None
    prev_team = None
    
    for i, pass_ in group.iterrows():
        current_player = pass_['playerId']
        current_team = pass_['uniqueTeamId']
        
        # Check for consecutive passes within the same team
        if prev_team == current_team and prev_player is not None:
            team_graphs[current_team].add_edge(prev_player, current_player)
        
        # Update the previous player and team for the next iteration
        prev_player = current_player
        prev_team = current_team
    
    # Calculate connectivity for each team graph
    connectivities = {}
    for team_id, graph in team_graphs.items():
        if len(graph) == 0:  # Handle cases where a team made no passes or only incomplete passes
            connectivity = 0
        else:
            connectivity = nx.average_node_connectivity(graph)
        connectivities[team_id] = connectivity
    
    return connectivities

In [39]:
test_dfdf = passes_df.iloc[:1000]

In [40]:
connectivity_results = [calculate_pass_connectivity(group) for _, group in passes_df.groupby(['matchId'])]

In [41]:
connectivity_results

[{'1609---2499719': 5.512820512820513, '1631---2499719': 1.511111111111111},
 {'1625---2499720': 3.878787878787879, '1651---2499720': 2.2666666666666666},
 {'1646---2499721': 1.0, '1610---2499721': 3.977777777777778},
 {'1628---2499722': 3.290909090909091, '1673---2499722': 1.6222222222222222},
 {'1623---2499723': 4.0181818181818185, '1639---2499723': 2.1777777777777776},
 {'1633---2499724': 3.018181818181818, '1611---2499724': 3.3454545454545452},
 {'1624---2499725': 5.6909090909090905, '1613---2499725': 1.0},
 {'1619---2499726': 3.9358974358974357, '10531---2499726': 1.8888888888888888},
 {'1644---2499727': 4.133333333333334, '1612---2499727': 2.7454545454545456},
 {'1659---2499728': 4.890909090909091, '1627---2499728': 1.4727272727272727},
 {'1659---2499729': 3.607142857142857, '1644---2499729': 2.0545454545454547},
 {'1646---2499730': 4.290909090909091, '1627---2499730': 1.1428571428571428},
 {'1673---2499731': 1.709090909090909, '1613---2499731': 4.072727272727272},
 {'1651---2499

In [42]:
connectivity_df = [(k, v) for d in connectivity_results for k, v in d.items()]
connectivity_df = pd.DataFrame(connectivity_df, columns=['uniqueTeamId', 'Connectivity'])

In [43]:
connectivity_df.set_index('uniqueTeamId', inplace=True)
connectivity_df

,Connectivity
uniqueTeamId,
1609---2499719,5.512821
1631---2499719,1.511111
1625---2499720,3.878788
1651---2499720,2.266667
1646---2499721,1.000000
...,...
3315---2576336,3.257576
3164---2576337,4.355556
3204---2576337,3.545455


In [44]:
final_df = pd.concat([motif_ratios, final_pass_types_df, pass_direction_df, shot_df_final, connectivity_df], axis=1)
# Iterate through each column in final_df to find and adjust the specified columns
for col in final_df.columns:
    # Check if the column name contains 'Backward', 'Forward', or 'Side'
    if any(keyword in col for keyword in ['Backward', 'Forward', 'Side']):
        # Divide the column by the 'Total_Passes' column
        final_df[col] = final_df[col] / final_df['Total_Passes']
        # Rename the column to add " Ratio" at the end
        final_df.rename(columns={col: col + " Ratio"}, inplace=True)

In [45]:
final_df

,ABAB,ABCA,ABCB,ABCD,Total_Passes,High_pass_Ratio_Total,Low_pass_Ratio_Total,High_pass_Ratio_Zone2,Low_pass_Ratio_Zone2,High_pass_Ratio_Zone3,...,Side Pass_Zone_2 Ratio,Side Pass_Zone_3 Ratio,Backward Pass Ratio,Forward Pass Ratio,Side Pass Ratio,Total Shots,Far Shots Ratio,Middle Distance Shots Ratio,Near Shots Ratio,Connectivity
uniqueTeamId,,,,,,,,,,,,,,,,,,,,,
10531---2499726,0.000000,0.108108,0.108108,0.783784,40,0.075000,0.925000,0.000000,1.000000,0.000000,...,0.350000,0.050000,0.125000,0.375000,0.500000,NaN,NaN,NaN,NaN,1.888889
10531---2499736,0.034483,0.086207,0.086207,0.793103,61,0.196721,0.786885,0.095238,0.904762,0.368421,...,0.229508,0.196721,0.049180,0.360656,0.590164,1.0,0.000000,1.000000,0.000000,3.015152
10531---2499741,0.030303,0.060606,0.060606,0.848485,36,0.333333,0.666667,0.222222,0.777778,0.000000,...,0.111111,0.027778,0.166667,0.583333,0.250000,1.0,1.000000,0.000000,0.000000,1.848485
10531---2499757,0.055556,0.088889,0.100000,0.755556,93,0.118280,0.870968,0.121212,0.878788,0.125000,...,0.161290,0.236559,0.129032,0.354839,0.516129,NaN,NaN,NaN,NaN,4.378788
10531---2499766,0.000000,0.000000,0.000000,1.000000,11,0.272727,0.727273,0.333333,0.666667,0.500000,...,0.090909,0.181818,0.090909,0.454545,0.454545,NaN,NaN,NaN,NaN,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756---2565880,0.034483,0.103448,0.103448,0.758621,90,0.211111,0.755556,0.142857,0.857143,0.343750,...,0.211111,0.222222,0.122222,0.355556,0.522222,3.0,0.000000,1.000000,0.000000,3.848485
756---2565892,0.015625,0.093750,0.078125,0.812500,67,0.208955,0.761194,0.151515,0.787879,0.388889,...,0.179104,0.164179,0.119403,0.388060,0.492537,3.0,0.333333,0.333333,0.333333,3.121212
756---2565904,0.037037,0.172840,0.172840,0.617284,84,0.190476,0.797619,0.131579,0.842105,0.444444,...,0.202381,0.119048,0.214286,0.321429,0.464286,3.0,0.000000,1.000000,0.000000,5.218182


In [46]:

if half is not None:
    final_df.to_csv(f'{table_name}_{half}_{min_min}_{max_min}.csv', index=True)
else:
    final_df.to_csv(f'{table_name}.csv', index=True)

In [47]:
half

'2H'

# END OF FILE